Your boss believes that NO2 tends to export more electricity to Germany whenever the
electricity price in Germany is higher relative to the price in NO2. She has asked you to
investigate this claim by creating a **scatter plot between the weekly sum of net exports in NO2
and the weekly average price difference between Germany and NO2.** Note that to find the
weekly average price difference, you should first find the hourly price difference, and then take
the weekly average.
The graph should:
- contain a single scatter plot that shows the weekly sum of net exports in NO2 on the y-
axis and the weekly average price difference between Germany and NO2 on the x-axis.
- contain the correlation coefficient between the weekly sum of net exports and the
weekly average price difference in the figure title.
- be stored as a png file called “figure_task5.png”.
What is your conclusion? Does there seem to be a relationship between the weekly sum of net
exports and the weekly average price difference? If so, is the relationship positive or negative?

In [1]:
#Global Variables
no2_data_path = 'data/PhysicalFlows_12.1.G'
no2_file_matter = 'PhysicalFlows_12.1.G.csv'
delu_data_path = 'data/DayAheadPrices_12.1.D'
delu_file_matter = 'DayAheadPrices_12.1.D.csv'
nordlink_opening = datetime(2020, 12, 9)

#NO2 Weekly Net Export, for Physical Flows
data = total_filtered_data(no2_data_path, no2_file_matter)
physical_data = data['physical_data']
period = 'week'
purpose = 'net'

#DE_LU Weekly Average Price, for Price
data = total_filtered_data(delu_data_path, delu_file_matter)
pricing_data = data['price_data']

"""
The pricing_export_correlation function provides insight into relationship between export patterns and price differentials for two defined regions.

Parameters:
    pricing_data (pd.DataFrame): DataFrame containing electricity prices, with 'DateTime' and 'MapCode' columns.
    physical_data (pd.DataFrame): DataFrame containing physical flow data with 'DateTime', 'InMapCode', 'OutMapCode', and 'FlowValue' columns.
    mapcode1 (str): The 'MapCode' representing the first region (e.g., 'DE_LU' for Germany).
    mapcode2 (str): The 'MapCode' representing the second region (e.g., 'NO2' for Norway).

Functionality:
1. Calculates Weekly Average Price Difference:
2. Calculates Weekly Net Export leveraging physical_flow function from Task 3
3. Merges Weekly Net Export and Weekly Average Difference
4. Computes the Correlation for these two metrics.

Returns:
    pd.DataFrame: A merged DataFrame containing the weekly average price difference and weekly net exports.
    float: The correlation coefficient between weekly net exports and weekly average price difference.

Reusability:
    - physical_flow function is reused making pricing_export_correlation easier to adapt
"""


def pricing_export_correlation(pricing_data, physical_data, mapcode1, mapcode2):
    #Isolating Necessary Columns Directly
    region1_prices = pricing_data[pricing_data['MapCode'] == mapcode1][['DateTime', 'Price']].set_index('DateTime')
    region2_prices = pricing_data[pricing_data['MapCode'] == mapcode2][['DateTime', 'Price']].set_index('DateTime')

    price_difference = region1_prices['Price'].subtract(region2_prices['Price'], fill_value=0)
    weekly_avg_price_difference = price_difference.resample('W').mean().reset_index()
    weekly_avg_price_difference.columns = ['Date', 'Weekly Avg Price Difference']

    weekly_net_export_df = physical_flow(physical_data, period, purpose)

    pricing_export_df = pd.merge(weekly_net_export_df, weekly_avg_price_difference, on='Date').dropna()

    if 'Net Export' in pricing_export_df.columns and 'Weekly Avg Price Difference' in pricing_export_df.columns:
        correlation = pricing_export_df['Net Export'].corr(pricing_export_df['Weekly Avg Price Difference'])
    else:
        print("Error: Required columns for correlation calculation are missing.")
        correlation = None
    
    return pricing_export_df, correlation

pricing_export_df, correlation = pricing_export_correlation(pricing_data, physical_data, 'DE_LU', 'NO2')

#Plotting
plt.figure(figsize=(10, 6))
plt.scatter(pricing_export_df['Weekly Avg Price Difference'], pricing_export_df['Net Export'], alpha=0.4)
plt.title(f"Weekly Net Exports vs. Weekly Avg Price Difference\nCorrelation Coefficient: {correlation:.2f}")
plt.xlabel("Weekly Average Price Difference (DE_LU - NO2) [EUR/MWh]")
plt.ylabel("Weekly Sum of Net Exports (NO2 to DE_LU) [MWh]")
plt.grid(True)
plt.savefig("figure_task5.png")
plt.show()

NameError: name 'datetime' is not defined